<a href="https://colab.research.google.com/github/carlos-alves-one/-ML-Zoomcamp-HW-4/blob/main/ML_Zoomcamp_HW_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Machine Learning ZoomCamp - Homework 3 
**Author.....: Carlos Manuel de Oliveira Alves**<br>
**Created...: 02/10/2022**

In [ ]:
# Import all necessay libraries that we will use in this homework
%matplotlib inline

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Import the library warnings to ignore the warnings from the system
import warnings
warnings.filterwarnings("ignore")